# OpenGridMap Notebook -- Data Preparation

Before executing any code in this Notebook, you should 
1. make sure that you have installed necessary tools 
2. prepare a database gis in your local PostGis server. 
Please look at Installation.ipynb for more details.

The goal of this notebook is to
1. Get farmilliar with the dataset
2. Learn APIs of PostGis and ipyleaflet 
3. Fuse required data into one or two compact tables

# Step1. Preparation

## Import necessary modules

In [1]:
import psycopg2
import json
import ipyleaflet as L
import networkx as nx
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Database connection

In [2]:
# Set connection to gis db
try:
    conn = psycopg2.connect("dbname=gis user=jennyzhou")
except:
    print 'Fail to connect to Postgres Server'

# Step2. Data selection

## Select area of Garching on map

In [3]:
# Select polygon of Garching
cur = conn.cursor() 
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326))), ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326))) FROM planet_osm_polygon WHERE osm_id = -30971; ")
garching = cur.fetchall()
cur.close()

garchingCentJson = json.loads(garching[0][0])
garchingPolJson = json.loads(garching[0][1])

## Select transformer entries

In [4]:
cur = conn.cursor()
# Select all the transformers inside Garching
cur.execute("SELECT ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(way, 4326)))) FROM student_transformers;")
transformers = cur.fetchall()
cur.close()
transJson = json.loads(transformers[0][0])

## Voronoi segmentation within Garching

In [5]:
cur = conn.cursor()
# Get voronoi diagram
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(ST_VoronoiPolygons(ST_Collect(way)), 4326))) FROM student_transformers;")
#cur.execute("SELECT ST_AsGeoJSON(ST_VoronoiPolygons(ST_Collect(ST_FlipCoordinates(ST_Transform(way, 4326))))) FROM student_transformers;")
voronoi = cur.fetchall()
voronoiJson = json.loads(voronoi[0][0])

## Select house entries

In [7]:
# cur = conn.cursor()
# # Too Slow
# cur.execute("SELECT ST_AsGeoJson(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326))) FROM planet_osm_polygon WHERE ST_Within(way,(SELECT way FROM planet_osm_polygon WHERE osm_id = -30971)) AND building != '' LIMIT 10; ")
# houses = cur.fetchall()
# cur.close()

# Step3. Data visualisation with ipyleaflet

## 3.1 General display with all data

In [6]:
# Initial map container with Garching as center
map = Map(default_tiles=TileLayer(opacity=1.0),center= garchingCentJson['coordinates'],zoom =13)
map

### Mark boundary of Garching

In [7]:
# Show boundary of Garching
garchingOuter = Polygon(locations=garchingPolJson['coordinates'], weight=4,
            color='#003d99', opacity=0.8, fill_opacity=0.0,
            fill_color='#ccffcc')
map += garchingOuter

### Voronoi Diagram

In [8]:
for poly in voronoiJson['geometries']:
    pg = Polygon(locations=poly['coordinates'], weight=2,
            color='#a3c2c2', opacity=0.8, fill_opacity=0.2,
            fill_color='#ccffcc') 
    map.add_layer(pg)

### Mark transformers

In [9]:
# Mark transformers
tmarkers = []
for tp in transJson['coordinates']:  
    mk = Marker(location=tp)
    tmarkers.append(mk)
    map.add_layer(mk)

In [175]:
# Clean Markers
for mk in tmarkers:
    map.remove_layer(mk)

## 3.2 Testing within one voronoi polygon

In [10]:
# Fix area -> good choice: 38 (or 37) 
tArea = voronoiJson['geometries'][38]

In [11]:
# Find corresponding transformers
tGeoJson = json.dumps(tArea)
cur = conn.cursor()
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326))) FROM student_transformers WHERE ST_Within(ST_FlipCoordinates(ST_Transform(way, 4326)),ST_SetSRID(ST_GeomFromGeoJSON(\'%s\'),4326));" % tGeoJson)
tTrans = cur.fetchall()

# Find corresponding houses
cur.execute("SELECT ST_AsGeoJson(ST_Collect(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326)))) FROM planet_osm_polygon WHERE ST_Within(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326)),ST_SetSRID(ST_GeomFromGeoJSON(\'%s\'),4326)) AND building != '';" % tGeoJson)
houses = cur.fetchall()
cur.close()
tTransJson = json.loads(tTrans[0][0])
tHouseJson = json.loads(houses[0][0])


In [12]:
# Mark houses
hmarkers = []
for row in houses:
    c = Circle(location=json.loads(row[0])['coordinates'], weight=5, opacity = 0.5, color = 'red', radius = 2) 
    hmarkers.append(c)
    map.add_layer(c) 

In [13]:
# Mark transformer
tCenter = tTransJson['coordinates']
tMarker =  Marker(location=tCenter)

# Mark polygon
tPoly = Polygon(locations=tArea['coordinates'], weight=2,
            color='#a3c2c2', opacity=0.8, fill_opacity=0.2,
            fill_color='#ccffcc') 

# Init Map
tMap = Map(default_tiles=TileLayer(opacity=1.0),center=tCenter ,zoom=17)
tMap.add_layer(tMarker)
tMap.add_layer(tPoly)
tMap

In [14]:
# Mark houses
hmarkers = []
for hp in tHouseJson['coordinates']:
    c = Circle(location= hp, weight=5, opacity = 0.5, color = 'red', radius = 2) 
    hmarkers.append(c)
    tMap.add_layer(c) 

In [ ]:
for mk in hmarkers:
    tMap.remove_layer(mk)

# Final Step. Fusion into new data tables

### Execute following section ONLY WHEN one needs to initialize all the tables!


In [297]:
cur = conn.cursor()
"""
# Store vornoi diagram into table vornoi
cur.execute("SELECT ST_AsGeoJSON(ST_VoronoiPolygons(ST_Collect(way))) FROM student_transformers;")
vor = cur.fetchall()
vorJson = json.loads(vor[0][0])
for geo in vorJson['geometries']:
    geoJson = json.dumps(geo)
    cur.execute("INSERT INTO vornoi (way) VALUES (ST_SetSRID(ST_GeomFromGeoJSON(\'%s\'),900913));" % geoJson)
    conn.commit()
    
# Mapping corresponding transformer
cur.execute("UPDATE vornoi SET tid = t.transformer_id, tway = t.way FROM student_transformers AS t WHERE ST_Within(t.way, vornoi.way);")
conn.commit()

# Storing mapping betweent houses and transformers into table vornoi_map
cur.execute("INSERT INTO vornoi_map (vid,oid,way,istransformer) SELECT v.vid, h.osm_id, ST_Centroid(h.way),false FROM vornoi AS v, planet_osm_polygon AS h WHERE ST_Within(ST_Centroid(h.way),v.way) AND building != '';")
conn.commit()

# Insert corresponding transformers into table vornoi_map for convenience
cur.execute("INSERT INTO vornoi_map (vid,oid,way,istransformer) SELECT v.vid, v.tid, v.tway, true FROM vornoi As v;")
conn.commit()
"""
cur.close()